## Lets build a NLP classification model for news articles using the AG News Dataset. This will classifiy if a news article is a World, Sports, Business or Sci/Tech Topic article.

In [1]:
import torch
import torchtext
from torchtext.datasets import text_classification
import os

## Get Data from the PyTorch Text Classification [Datasets](https://pytorch.org/text/datasets.html?highlight=textclassification). We are using the AG News dataset for this model

In [2]:
# check directory for data if it doesnt already exist
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
    
#Get train and text dataset to tensor
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=2, vocab=None)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

120000lines [00:09, 12466.07lines/s]
120000lines [00:16, 7295.77lines/s]
7600lines [00:01, 7593.33lines/s]


In [3]:
train_dataset[10000]

(0,
 tensor([     6,   2281,    819,     22,     64,      2,     10,      2,     93,
            372,   2926,     14,     36,     15,     36,     16,     21,      7,
             76,      4,   1591,      2,    934,      4, 535870,     64,      2,
             10,      2,    214,   1054,     49,   1051,    340,      3,   2444,
              7,    372,   1210,      8,     93,      8,   2192,   1610,      4,
            287,      5,      3,    698,    772,      2,      7,    945,      4,
         532969,   1051,     21,      6,   1972,      7,   2150,    875,      9,
          94748,   1051,      7,  90431,   9903,      2,  48627,  87577,   3200,
          10519,     65,     73,     68,  46053,  99671,  36228,  42433,     62,
             63,     71,     70,   4627,  10756,  33697,    896,   3357,   2482,
           4687,   7813, 491902, 535871,     65,     73,     68, 141587,  83349,
          42944,   8042,  33312,   1849,   7724,   8380,  24392,  31424,   7653,
            274,  12894,

In [4]:
train_dataset.get_vocab()

In [6]:
train_dataset.get_labels()

{0, 1, 2, 3}

##  Construct an Optimizer, an iterable containing the parameters to optimize. Then, you can specify optimizer-specific options such as the learning rate, weight decay, etc.

Optimizers are algorithms or methods used to change the attributes of the neural network such as weights and learning rate to reduce the losses.

Basically the optimizer tells us how bad we are doing after each layer.

In [12]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


## Functions used to generate batch

Since the text entries have different lengths, a custom function generate_batch() is used to generate data batches and offsets. The function is passed to collate_fn in torch.utils.data.DataLoader. The input to collate_fn is a list of tensors with the size of batch_size, and the collate_fn function packs them into a mini-batch.

In [8]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

## Define functions to train the model and evaluate results.

In [9]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [13]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
BATCH_SIZE = 16
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

## Train the Model

CrossEntropyLoss criterion combines nn.LogSoftmax() and nn.NLLLoss() in a single class. It is useful when training a classification problem with C classes. SGD implements stochastic gradient descent method as optimizer. The initial learning rate is set to 4.0. StepLR is used here to adjust the learning rate through epochs.


Softmax is an activation function of a neural network to normalize the output of a network to a probability distribution over predicted output classes.

In [14]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

#activation function
criterion = torch.nn.CrossEntropyLoss().to(device)
#Stochastic Gradient descient with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 22 seconds
	Loss: 0.0262(train)	|	Acc: 84.7%(train)
	Loss: 0.0001(valid)	|	Acc: 90.1%(valid)
Epoch: 2  | time in 0 minutes, 22 seconds
	Loss: 0.0119(train)	|	Acc: 93.7%(train)
	Loss: 0.0001(valid)	|	Acc: 90.8%(valid)
Epoch: 3  | time in 0 minutes, 22 seconds
	Loss: 0.0069(train)	|	Acc: 96.4%(train)
	Loss: 0.0002(valid)	|	Acc: 90.2%(valid)
Epoch: 4  | time in 0 minutes, 22 seconds
	Loss: 0.0039(train)	|	Acc: 98.1%(train)
	Loss: 0.0001(valid)	|	Acc: 90.7%(valid)
Epoch: 5  | time in 0 minutes, 22 seconds
	Loss: 0.0023(train)	|	Acc: 99.0%(train)
	Loss: 0.0001(valid)	|	Acc: 91.0%(valid)


# Test with testing dataset

In [15]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0003(test)	|	Acc: 88.0%(test)


## Test with String text

In [16]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news


In [17]:
predict(ex_text_str, model, vocab, 2)

2

Resources:
This example is from the [PyTorch Beginner Tutorial](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html)